In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd

In [2]:
url = 'https://finance.naver.com/sise/sise_market_sum.nhn?sosok=0&page='
headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36'}

## 네이버 주가 금융 스크래핑

- csv 파일로 저장

In [4]:
filename = '시가총액 1-200.csv'
f = open(filename, 'w', encoding = 'utf-8-sig', newline = '')  # newline 옵션을 넣지 않을 경우 행마다 공백이 생긴다.
writer = csv.writer(f)

title = 'N	종목명	현재가	전일비	등락률	액면가	시가총액	상장주식수	외국인비율	거래량	PER	ROE'.split('\t') # split('\t') : 탭으로 구분
writer.writerow(title)

for page in range(1, 5):
    res = requests.get(url + str(page), headers = headers)
    soup = BeautifulSoup(res.text, 'lxml')
    
    # table에서 class 속성값이 type_2인 것 중 tbody아래의 tr의 모든 데이터를 수집
    data_rows = soup.find('table', attrs={'class' : 'type_2'}).find('tbody').find_all('tr')
    
    for row in data_rows:
        columns = row.find_all('td')
        if len(columns) <=1:               # td 내용 중 컬럼의 개수가 1개 이하인 의미없는 데이터는 제거한다.
            continue                      # 저장하지 않고 계속
        data = [column.get_text() for column in columns]
        # 한줄 for 문(아래와 출력은 동일함)
        # for column in columns:
            #data = column.get_text()

        writer.writerow(data)

### 바로가는 링크 삽입

In [5]:
filename = '시가총액 1-200.csv'
f = open(filename, 'w', encoding = 'utf-8-sig', newline = '')  # newline 옵션을 넣지 않을 경우 행마다 공백이 생긴다.
writer = csv.writer(f)

title = 'N	종목명	현재가	전일비	등락률	액면가	시가총액	상장주식수	외국인비율	거래량	PER	ROE	URL'.split('\t') # split('\t') : 탭으로 구분
writer.writerow(title)

for page in range(1, 5):
    res = requests.get(url + str(page), headers = headers)
    soup = BeautifulSoup(res.text, 'lxml')
    
    # table에서 class 속성값이 type_2인 것 중 tbody아래의 tr의 모든 데이터를 수집
    data_rows = soup.find('table', attrs={'class' : 'type_2'}).find('tbody').find_all('tr')
    
    for row in data_rows:
        columns = row.find_all('td')
        if len(columns) <=1:               # td 내용 중 컬럼의 개수가 1개 이하인 의미없는 데이터는 제거한다.
            continue                      # 저장하지 않고 계속
        data = [column.get_text() for column in columns]
        # 한줄 for 문(아래와 출력은 동일함)
        # for column in columns:
            #data = column.get_text()
        link = data.append('https://finance.naver.com/' + row.find('a', attrs = {'class' : 'tltle'})['href'])
        if link:
            data = data.append(link)
        data.remove('')                    # 데이터 공백값 제거
        
        writer.writerow(data)

## 최종 버전

In [3]:
data=[]
link=[]

for page in range(1,5):
    res = requests.get(url + str(page), headers = headers)
    soup = BeautifulSoup(res.text, 'lxml')    # lxml로 분석을 한다? <- 이 내용에 대해 차후 알아봐야 할 듯
    
    data_row = soup.find('table', attrs = {'class' : 'type_2'}).find('tbody').find_all('tr')
    
    for row in data_row:
        column = row.find_all('td')
        if len(column) <= 1:
            continue
        data.append([column.get_text() for column in column])
        link.append('https://finance.naver.com/' + row.find('a', attrs = {'class' : 'tltle'})['href'])
        
title = 'N	종목명	현재가	전일비	등락률	액면가	시가총액	상장주식수	외국인비율	거래량	PER	ROE	URL'.split('\t') # split('\t') : 탭으로 구분
data = pd.DataFrame(data)
data = data.iloc[:,0:12]

link = pd.DataFrame(link)
data = pd.concat([data,link],axis=1)
data.columns = title

data.to_csv('C:/Users/Park/_Python_data/시가총액_200.csv', encoding = 'utf-8-sig',index=False)